### 구글 코랩을 이용함
https://drive.google.com/file/d/1LLX9WZLl0X4nrB7KqVDXkkCKl5o_Mccs/view?usp=sharing

# 6. KBBert을 활용한 ESG 등급 예측모델
- GPU로 Fine-tuning KB-ALBERT with Transformers TensorFlow
1. 데이터 준비 & 필요 라이브러리 다운로드
2. Tokenizer를 활용한 학습용 데이터셋 생성
3. 학습 하이퍼파라미터 설정
4. Bigkinds의 특성추출을 활용한 모델 Fine-tuning
5. Inference를 위한 Pipeline 생성

## 6.1. 데이터 준비 & 필요 라이브러리 다운로드

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install -q transformers

In [ ]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


---

## tone fine tunning

In [ ]:
# 1) esg label data csv 불러오기
import pandas as pd
# df_final1 = pd.read_csv('/content/drive/MyDrive/data/df_final_tone11.csv')
# df_final1.drop('Unnamed: 0',axis=1,inplace=True)
# df_final2 = pd.read_csv('/content/drive/MyDrive/data/df_final_tone21.csv')
# df_final2.drop('Unnamed: 0',axis=1,inplace=True)

df_final = pd.read_csv('/content/drive/MyDrive/df_final_tone.csv')
df_final.drop('Unnamed: 0.1',axis=1,inplace=True)
df_final.drop('Unnamed: 0',axis=1,inplace=True)
df_final

,제목,특성추출(가중치순 상위 50개),본문,본문_번역,label,score
0,"환경부, 2050 탄소중립 담은 LEDS 영문본 30일 UN에 제출","사무국,영문본,탄소중립,배출량,유엔기후변화협약사무국,국무회의,장기저탄소,장기저탄소발...",환경부는 유엔기후변화협약사무국의 국가연락책임관인 외교부를 통해 '2030 국가온실가...,The Ministry of Environment announced on the 3...,Neutral,0.999894
1,코로나19 타격 주름살 깊었던 전북경제,"일자리,코로나19,실수요자,박웅,전라북도,전북경제,도민들,군산조선소,현대중공업,중국...",[KBS 전주]\n [앵커]\n\n 전북경제도 그 어느 때보다도 힘든 시간을 보냈습...,[KBS Jeonju]\n [anchor]\n\n The Jeollabuk-do e...,Negative,0.999996
2,"한은이 '고용안정'도 책임져라? 이주열 ""정책 일관성 유지 어렵다"" 난색","한은,중앙은행,통화정책,이주열,한은법,연준,개정안,코로나19,한국은행,미국,신년사,...",‘물가 안정’과 ‘금융 안정’을 설립 목표로 하고 있는 한국은행에 ‘고용 안정’ 책...,Bank of Korea Governor Lee Ju-yeol expressed a...,Negative,0.999854
3,"집값은 안 잡히고 선거는 다가오고 마음 급한 정부, 청년층 공략 나서","고시원,국토부,기숙사,청년특화주택,청년층,감독관,월평균소득,보증금,공급대책,전세금,...",[디지털타임스 박상길 기자] 정부가 청년층을 위해 2025년까지 도심에 청년특화주택...,[Digital Times Reporter Park Sang-gil] The gov...,Neutral,0.999869
4,"이성 구로구청장 ""‘해고없는 도시’ 만들기 위해 고용유지지원금과 고용보험료 지원""","구청장,코로나,지원금,구로구청장,청년이룸,무상급식,취업박람회,신년사,활성화,대한민국...","[아시아경제 박종일 기자] 이성 구로구청장은 ""구는 올해도 구정의 제1목표를 ‘코로...",[Asia Economy Reporter Park Jong-il] Lee Seong...,Neutral,0.920470
...,...,...,...,...,...,...
24506,변화 주저하지 않고 위기때 재빠르게 대처 쥐띠 CEO가 뛴다,"쥐띠,경영인,상장사,현대백화점그룹,삼성전자,최태원,전문경영인,허창수,태스크포스,명예...","최태원 SK그룹 회장, 이재현 CJ그룹 회장, 정지선 현대백화점그룹 회장 등의 공통...","What do SK Group Chairman Chey Tae-won, CJ Gro...",Positive,1.000000
24507,[인투더퓨처-클라우드] 2020년 '클라우드 온리' 시대 열린다,"클라우드,계열사,경영진,카카오뱅크,클라우드로,erp,삼성그룹,sk,대기업,현대기아차",기업 경영에 클라우드 도입을 당연하게 여기는 '클라우드 온리(Cloud Only)'...,"The era of 'Cloud Only', which takes cloud ado...",Positive,0.995308
24508,[인사] 국민일보 外,"사업단,지점장,팀장,센터장,본부장,서울,부서장,지사장,부산,부동산,강남,지원부장,분...",◆국민일보◇편집국△금융전문기자 이동훈△화상팀 사진DB담당 최병희△산업부 차장 강주화...,◆Kookmin Ilbo ◇ Editorial Office △ Financial C...,Neutral,0.999975
24509,CJ가 만드는 미래..'즐거운동행' 진행 [CJ의 나눔경영],"햇반,제일제당,cj제일제당,운동행,경쟁력,협력사,동반성장,아산시,위원회,하도급,중소...",경쟁력 있는 협력사 발굴 자금 판로 등 지원 \n중기 핵심역량 활용한 제품 출시 동...,"Support for finding competitive partners, fund...",Positive,0.999992


In [ ]:
# # 2) csv concat으로 합치기
# df_final = pd.concat([df_final1,df_final2])
# df_final.reset_index(drop=True,inplace=True)
# df_final.drop('Unnamed: 0.1',axis=1,inplace=True)
# df_final

In [ ]:
# 3) label 빈도수 체크
df_final.groupby('label').count()

,제목,특성추출(가중치순 상위 50개),본문,본문_번역,score
label,,,,,
Negative,3202,3202,3202,3202,3202
Neutral,16099,16099,16099,16099,16099
Positive,5210,5210,5210,5210,5210


In [ ]:
df_final_tone = df_final.copy()

In [ ]:
# 5) df_final_tone 전처리
df_final_tone['본문'] = df_final_tone['본문'].apply(str)
df_final_tone['label'] = df_final_tone['label'].replace(["Neutral","Positive","Negative"],[0,1,2])
df_final_tone = df_final_tone.dropna(subset=['본문', 'label'])
df_final_tone = df_final_tone[['본문','label']]
df_final_tone

,본문,label
0,환경부는 유엔기후변화협약사무국의 국가연락책임관인 외교부를 통해 '2030 국가온실가...,0
1,[KBS 전주]\n [앵커]\n\n 전북경제도 그 어느 때보다도 힘든 시간을 보냈습...,2
2,‘물가 안정’과 ‘금융 안정’을 설립 목표로 하고 있는 한국은행에 ‘고용 안정’ 책...,2
3,[디지털타임스 박상길 기자] 정부가 청년층을 위해 2025년까지 도심에 청년특화주택...,0
4,"[아시아경제 박종일 기자] 이성 구로구청장은 ""구는 올해도 구정의 제1목표를 ‘코로...",0
...,...,...
24506,"최태원 SK그룹 회장, 이재현 CJ그룹 회장, 정지선 현대백화점그룹 회장 등의 공통...",1
24507,기업 경영에 클라우드 도입을 당연하게 여기는 '클라우드 온리(Cloud Only)'...,1
24508,◆국민일보◇편집국△금융전문기자 이동훈△화상팀 사진DB담당 최병희△산업부 차장 강주화...,0
24509,경쟁력 있는 협력사 발굴 자금 판로 등 지원 \n중기 핵심역량 활용한 제품 출시 동...,1


In [ ]:
# 1) train_test_split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_final_tone, stratify=df_final_tone['label'], test_size=0.2, random_state=44)
print(df_train.shape, df_test.shape)

(19608, 2) (4903, 2)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 26.8 MB/s 
     |████████████████████████████████| 6.6 MB 15.2 MB/s 
     |████████████████████████████████| 120 kB 47.4 MB/s 


In [ ]:
# 2) Custom Tokenizer 불러오기, warning message 보여주지 않기
from transformers import AutoModel, AutoTokenizer
from transformers import logging
import torch
logging.set_verbosity_warning()

kb_albert_model_path = "/content/drive/MyDrive/kb-albert-char-base-v2/kb-albert-char-base-v2"
model = AutoModel.from_pretrained(kb_albert_model_path)  # Load Model from pytorch checkpoint
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)

Some weights of the model checkpoint at /content/drive/MyDrive/kb-albert-char-base-v2/kb-albert-char-base-v2 were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 3) df_train, df_test 전처리
df_train['본문'] = df_train['본문'].str.replace(',',' ')
df_test['본문'] = df_test['본문'].str.replace(',',' ')

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 32.2 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 115 kB 68.8 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
# 4) Tokenizer을 통해 raw test를 전처리
# 5) pytorch dataset으로 학습용 데이터셋 생성
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['본문'], truncation=True, padding='max_length'), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['본문'], truncation=True, padding='max_length'), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import numpy as np

args = TrainingArguments(
        output_dir = './results', # output이 저장될 경로
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=16, # 각 device 당 batch size
        num_train_epochs=5, # train 시킬 총 epochs
        weight_decay=0.01, # weight decay
        logging_dir='./logs', # log가 저장될 경로
)

In [ ]:
# +a) 분류 예측 metrics 함수 만들기
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

In [ ]:
# 1) Fine-tuning을 위한 Head에 사전학습한 언어모델 불러오기
from transformers import AlbertForSequenceClassification

model = AlbertForSequenceClassification.from_pretrained(kb_albert_model_path,num_labels=3)

Some weights of the model checkpoint at /content/drive/MyDrive/kb-albert-char-base-v2/kb-albert-char-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initial

In [ ]:
%%time
trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 19608
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6130


Step,Training Loss
500,0.790200
1000,0.732200
1500,0.596700
2000,0.546600
2500,0.529900
3000,0.466900
3500,0.467400
4000,0.415100
4500,0.411800
5000,0.386400


Saving model checkpoint to ./results/checkpoint-1226
Configuration saved in ./results/checkpoint-1226/config.json
Model weights saved in ./results/checkpoint-1226/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2452
Configuration saved in ./results/checkpoint-2452/config.json
Model weights saved in ./results/checkpoint-2452/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3678
Configuration saved in ./results/checkpoint-3678/config.json
Model weights saved in ./results/checkpoint-3678/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-4904
Configuration saved in ./results/checkpoint-4904/config.json
Model weights saved in ./results/checkpoint-4904/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-6130
Configuration saved in ./results/checkpoint-6130/config.json
Model weights saved in ./results/checkpoint-6130/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 2h 5min 37s, sys: 15min, total: 2h 20min 38s
Wall time: 2h 20min 59s


TrainOutput(global_step=6130, training_loss=0.49913024715653836, metrics={'train_runtime': 8452.8044, 'train_samples_per_second': 11.599, 'train_steps_per_second': 0.725, 'total_flos': 2343201503109120.0, 'train_loss': 0.49913024715653836, 'epoch': 5.0})

In [ ]:
# Test data에 fine-tuned 모델 성능 평가하기
model.eval()
trainer.predict(dataset_test).metrics

***** Running Prediction *****
  Num examples = 4903
  Batch size = 8


{'test_loss': 0.4825667142868042,
 'test_accuracy': 0.8141953905771976,
 'test_runtime': 163.7279,
 'test_samples_per_second': 29.946,
 'test_steps_per_second': 3.744}

In [ ]:
from transformers import pipeline

tone_classifier = pipeline('sentiment-analysis', 
                           model=model.cpu(), tokenizer=tokenizer, framework='pt')
id1label = {"LABEL_0": "Neutural", "LABEL_1": "Positive","LABEL_2": "Negative"}
texts= [
    "소방서,가스측정방법,아산소방서,주식회사,환기방법,밀폐공간,훈련장,김성찬,대원,119구조,jb,아산,훈련,합동,구조,맨홀,대비,방법,직원,가정,옥상,기구,사고,장면,인명,환기,밀폐,측정,가스,서장,공간,질식",
    "확진자,서울,코로나19,수도권,접촉자,1명,리치웨이,탁구장,치명률,쿠팡물류센터,방대본,소모임,방문판매업체,50대,집단감염,4명,집단발병,이태원,입소자",
    "이재용,서울,리치웨이,코로나19,질본,다단계,관리청,박병석,본회의,삼성전자,확진자,주식회사,외부감사,주식회사등,영장실질심사,문재인,더불어민주당,감염병,500원,시장법,이기주의,연구원,중앙행정기관,삼성",
    "사무국,영문본,탄소중립,배출량,유엔기후변화협약사무국,국무회의,장기저탄소,장기저탄소발전전략,leds,환경부,국가온실가스감축목표,외교부,우리나라,책임관,국문본,7억,정책과제들,910만,관계부처,un,국가연락책임관,ndc,감축,국가,제출,전략",
    "공정위,롯데칠성,롯데지주,과징금,롯데그룹,롯데,심사보고서,자회사,그룹사,mja와인,롯데하이마트,보고서,전원회의,롯데마트,mja,공정거래위원회,공소장,유통업법,위원회,관계자",
    "하나은행,특별퇴직,40세,특별퇴직자,퇴직자,관리자급,책임자,책임자급,재채용,이날부,1인,일반직,효율화,3배,당사자,일시지급,000만,000만,500만,평균임금,재취업,학자금,지원금",
    "환경부,탄소중립,사무국,배출량,조명래,장기저탄소,외교부,책임관,우리나라,장기저탄소발전전략,정책과제들,저탄소,방향성,자신감,7억,저탄소발전전략,온실가스감축목표,국가온실가스감축목표,국가연락책임관,un,leds,ndc,국가,감축,전략,정부,중립,상향"
]

results = tone_classifier(texts)
for idx, result in enumerate(results):
    print(texts[idx])
    for k, v in result.items():
        print(f" >> {k} : {id1label[v] if k == 'label' else v}")
    print()

Disabling tokenizer parallelism, we're using DataLoader multithreading already


소방서,가스측정방법,아산소방서,주식회사,환기방법,밀폐공간,훈련장,김성찬,대원,119구조,jb,아산,훈련,합동,구조,맨홀,대비,방법,직원,가정,옥상,기구,사고,장면,인명,환기,밀폐,측정,가스,서장,공간,질식
 >> label : Neutural
 >> score : 0.9911803603172302

확진자,서울,코로나19,수도권,접촉자,1명,리치웨이,탁구장,치명률,쿠팡물류센터,방대본,소모임,방문판매업체,50대,집단감염,4명,집단발병,이태원,입소자
 >> label : Neutural
 >> score : 0.9910932183265686

이재용,서울,리치웨이,코로나19,질본,다단계,관리청,박병석,본회의,삼성전자,확진자,주식회사,외부감사,주식회사등,영장실질심사,문재인,더불어민주당,감염병,500원,시장법,이기주의,연구원,중앙행정기관,삼성
 >> label : Neutural
 >> score : 0.9892878532409668

사무국,영문본,탄소중립,배출량,유엔기후변화협약사무국,국무회의,장기저탄소,장기저탄소발전전략,leds,환경부,국가온실가스감축목표,외교부,우리나라,책임관,국문본,7억,정책과제들,910만,관계부처,un,국가연락책임관,ndc,감축,국가,제출,전략
 >> label : Neutural
 >> score : 0.9924083948135376

공정위,롯데칠성,롯데지주,과징금,롯데그룹,롯데,심사보고서,자회사,그룹사,mja와인,롯데하이마트,보고서,전원회의,롯데마트,mja,공정거래위원회,공소장,유통업법,위원회,관계자
 >> label : Neutural
 >> score : 0.9920058846473694

하나은행,특별퇴직,40세,특별퇴직자,퇴직자,관리자급,책임자,책임자급,재채용,이날부,1인,일반직,효율화,3배,당사자,일시지급,000만,000만,500만,평균임금,재취업,학자금,지원금
 >> label : Neutural
 >> score : 0.9905475974082947

환경부,탄소중립,사무국,배출량,조명래,장기저탄소,외교부,책임관,우리나라,장

In [ ]:
# save the fine-tuned model
trainer.save_model('kbalbert-tone/')

Saving model checkpoint to kbalbert-tone/
Configuration saved in kbalbert-tone/config.json
Model weights saved in kbalbert-tone/pytorch_model.bin


### 학습된 ESG와 TONE 모델은 1번_영문번역 및 label 붙이기 마지막 부분에서 사용됨.